In [ ]:
!pip install boto3 tqdm ultralytics roboflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model
model = YOLO("yolov8m.pt")

model.train(
    data="/content/drive/MyDrive/Combined_Dataset/dataset.yaml",
    epochs=300,
    batch=16,
    imgsz=640,
    device="cuda",
    lrf=0.0001,
    patience=0,
    #cache=True,
    workers=32
)

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/best.pt")

destination_path = "/content/drive/MyDrive/YOLOv8_Model/best.pt"

# Copy model to Drive
import shutil
shutil.copy("/content/runs/detect/train/weights/best.pt", destination_path)

print(f"Model successfully saved in: {destination_path}")

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Function to draw bounding boxes
def plot_image_with_boxes(image_path, label_path, class_names=["class_0", "class_1"]):

    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Error: The image was not found in the path: {image_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_height, img_width, _ = img.shape

    with open(label_path, 'r') as file:
        lines = file.readlines()

    colors = [(0, 255, 0), (255, 0, 0)]

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])  # ID de la clase
        x_center, y_center, width, height = map(float, parts[1:])

        # Convert normalized coordinates to pixels
        x_min = int((x_center - width / 2) * img_width)
        y_min = int((y_center - height / 2) * img_height)
        x_max = int((x_center + width / 2) * img_width)
        y_max = int((y_center + height / 2) * img_height)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), colors[class_id], 2)
        cv2.putText(img, class_names[class_id], (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_id], 2)

    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

image_path = "/content/drive/MyDrive/Combined_Dataset/images/test/test_438.jpg"
label_path = "/content/drive/MyDrive/Combined_Dataset/labels/test/test_438.txt"
plot_image_with_boxes(image_path, label_path, class_names=["product", "missed"])

In [ ]:
# Load the trained YOLOv8 model
model = YOLO("/content/drive/MyDrive/YOLOv8_Model/best.pt")

# Test image path
image_path = "/content/drive/MyDrive/Combined_Dataset/images/test/test_438.jpg"

# Make the prediction
results = model.predict(image_path, imgsz=640, conf=0.3)

image = cv2.imread(image_path)

class_colors = {
    0: (0, 255, 0),
    1: (0, 0, 255)
}

for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Box coordinates
        class_id = int(box.cls[0])  # Class ID
        conf = box.conf[0]  # Confidence of the prediction

        color = class_colors.get(class_id, (255, 255, 255))

        # Draw the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 3)
        cv2.putText(image, f'{result.names[class_id]} {conf:.2f}',
                    (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
